# Yeast cells detection sample notebook

In [11]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.visualize as visualize

In [12]:
cd ~

/home/p301081


### Load data

In [3]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Test_Images'
filenames = data.load_data(path, ff = '.png')
print(filenames[0])

../../slow-data/yeastcells/data/DATA_CIT/Movie_1/tiffs/12.4.2017_sfp1_ph-tdgfp_hta2_mcherry_1pc_glu_to_1pc_glu_rap_xy01.nd2.tif


In [4]:
fn = filenames[1]
image = data.read_image(fn, single_im = False)
np.shape(image) #(frames, length, width, channels)

(217, 512, 512, 3)

### Load and run model on data for segmentation

In [41]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [42]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image_
]

### Run clustering algorithm for tracking

In [43]:
#labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.4, progress=False) #for movies
labels, coordinates = features.get_instance_numbers(output) #for single image

In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Extract features and visualize tracking and segmentation

In [45]:
s, t = features.get_seg_track(labels, output, frame = None) # total

The number of segmentations is 100
The number of tracked cells is 0


In [47]:
s, t = features.get_seg_track(labels, output, frame = 1) # by frame

The number of segmentations in frame 1 is 100
The number of tracked cells in frame 1 is 0


In [48]:
features.track_len(labels, label_num=0) #select a label to extract the length of its track

0

In [49]:
contours = features.extract_contours(output) 

In [ ]:
%matplotlib notebook
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=False, labelnum=False)
movie = visualize.show_animation(scene, title='', delay=500) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(scene, coordinates, labels, w=40, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=500)

In [ ]:
polygons_clust =  features.polygons_per_cluster(contours, output)
ax = visualize.plot_area_profile(polygons_clust, label=5, ax=None, title=None)

In [51]:
_, _, pred_df = data.get_pred(output, labels, coordinates)
polygons_inst =  features.polygons_per_instance(contours, output)
pred_features = features.get_features_df(polygons_inst, labels, pred_df)